# Warming stripes with Galaxy and Bioblend

In [23]:
import os
import time
from urllib.parse import urljoin
import os
from collections import defaultdict

import bioblend.galaxy
import tempfile
import pooch
import json

In [28]:
def upload_to_history(inputs, gi, hist_hid):
    ret_uploads = []
    for input_filename in inputs:
        ret = gi.tools.upload_file(input_filename, hist_id)
        ret_uploads.append(ret)
    return ret_uploads

## Input parameters

In [29]:
server = "https://usegalaxy.eu/"
api_key = os.environ.get("GALAXY_API_KEY")
workflow_parameters_filename = "workflow_input_params.json"

## Read Workflow parameters from workflow_invocation.json

In [38]:
with open(workflow_parameters_filename, 'r', encoding='utf-8') as file:
            workflow_parameters = json.load(file)
print(workflow_parameters)

{'inputs': ['downloaded_workflows/datasets/ts_cities.csv_31e7840b5aedca433fb349714141a239.tabular'], 'workflow': 'downloaded_workflows/workflows/f1ada1d68e850ab0.ga', 'params': [{'title': 'My ScienceLive Stripes', 'variable': 'tg_anomalies_freiburg', 'adv|colormap': 'RdBu_r'}]}


## Connect to a Galaxy instance (here Galaxy Europe)

In [31]:
# Optional: Check if API key was found
if not api_key:
    raise ValueError("GALAXY_API_KEY environment variable not set")

print(f"API Key loaded: {'Yes' if api_key else 'No'}")

gi = bioblend.galaxy.GalaxyInstance(url=server, key=api_key)

API Key loaded: Yes


## Create a new history

In [32]:
new_hist = gi.histories.create_history(name='ScienceLive')
print(new_hist)

{'model_class': 'History', 'id': '7616c018d9d3c3f2', 'name': 'ScienceLive', 'deleted': False, 'purged': False, 'archived': False, 'url': '/api/histories/7616c018d9d3c3f2', 'published': False, 'count': 0, 'annotation': None, 'tags': [], 'update_time': '2025-05-25T19:50:11.500464', 'preferred_object_store_id': None, 'contents_url': '/api/histories/7616c018d9d3c3f2/contents', 'size': 0, 'user_id': '8a9aff0c11cb3a70', 'create_time': '2025-05-25T19:50:11.515047', 'importable': False, 'slug': None, 'username': 'annefou', 'username_and_slug': None, 'genome_build': None, 'state': 'new', 'state_ids': {'new': [], 'upload': [], 'queued': [], 'running': [], 'ok': [], 'empty': [], 'error': [], 'paused': [], 'setting_metadata': [], 'failed_metadata': [], 'deferred': [], 'discarded': []}, 'state_details': {'new': 0, 'upload': 0, 'queued': 0, 'running': 0, 'ok': 0, 'empty': 0, 'error': 0, 'paused': 0, 'setting_metadata': 0, 'failed_metadata': 0, 'deferred': 0, 'discarded': 0}}


## Import workflow to Galaxy Europe

In [34]:
wf = gi.workflows.import_workflow_from_local_path(workflow_parameters["workflow"])

In [53]:
wf_id = wf["id"]
print(wf_id)

0012799af6e2a4f1


## Upload data in the history

In [42]:
hist_id = new_hist["id"]
hist_id

'7616c018d9d3c3f2'

In [44]:
ret_uploads = upload_to_history(workflow_parameters["inputs"], gi, hist_id)

# Get outputs

In [50]:
step = 0
inputs = {}
for ret in ret_uploads:
    hda = ret['outputs'][0]
#    inputs = {idx: {'id': hda['id'], 'src': 'hda'}}
    inputs.update({str(step): {'id': hda['id'], 'src': 'hda'}})
print(inputs) # will not work when several inputs!

{'0': {'id': '4838ba20a6d86765dc4198cfd98f3904', 'src': 'hda'}}


In [52]:
step = 1
params = {
    str(step): workflow_parameters["params"][0]
}

print(params)

{'1': {'title': 'My ScienceLive Stripes', 'variable': 'tg_anomalies_freiburg', 'adv|colormap': 'RdBu_r'}}


In [54]:
ret = gi.workflows.invoke_workflow(wf_id, inputs=inputs, params=params, history_id=hist_id)

In [56]:
while True:
    history = gi.histories.show_history(hist_id, contents=True)
    states = [dataset['state'] for dataset in history]
    if all(state in ['ok', 'error'] for state in states):
        print("✅ Workflow has completed.")
        break
    else:
        print("⏳ Waiting for workflow to complete...")
        time.sleep(10)

⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
⏳ Waiting for workflow to complete...
✅ Workflow has completed.


## Get RO-Crate

In [57]:
response = gi.invocations.get_invocation_archive(
        invocation_id = ret["id"],
        model_store_format= "rocrate.zip")

In [58]:
file = "climate.rocrate.zip"

with open(file, "bw") as archive:
    for chunk in response.iter_content(chunk_size=8192):
        archive.write(chunk)
        # Verify file is not empty

# Delete history

In [59]:
gi.histories.delete_history(hist_id, purge=True)

{'model_class': 'History',
 'id': '7616c018d9d3c3f2',
 'name': 'ScienceLive',
 'deleted': True,
 'purged': True,
 'archived': False,
 'url': '/api/histories/7616c018d9d3c3f2',
 'published': False,
 'count': 2,
 'annotation': None,
 'tags': [],
 'update_time': '2025-05-25T20:11:24.351574',
 'preferred_object_store_id': None,
 'contents_url': '/api/histories/7616c018d9d3c3f2/contents',
 'size': 58721,
 'user_id': '8a9aff0c11cb3a70',
 'create_time': '2025-05-25T19:50:11.515047',
 'importable': False,
 'slug': None,
 'username': 'annefou',
 'username_and_slug': None,
 'genome_build': None,
 'state': 'ok',
 'state_ids': {'new': [],
  'upload': [],
  'queued': [],
  'running': [],
  'ok': ['4838ba20a6d86765dc4198cfd98f3904',
   '4838ba20a6d86765cec420ab4c7f27e4'],
  'empty': [],
  'error': [],
  'paused': [],
  'setting_metadata': [],
  'failed_metadata': [],
  'deferred': [],
  'discarded': []},
 'state_details': {'new': 0,
  'upload': 0,
  'queued': 0,
  'running': 0,
  'ok': 2,
  'empty':